In [ ]:
# Importando as bibliotecas que serão usadas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import awswrangler as wr

In [ ]:
# Importando a base de dados do S3

#df = wr.s3.read_csv(
#  path='s3://dataset_workshop.CSV',
#  sep=';',
#  encoding='latin-1'
#)


df = pd.read_csv('../Database/dataset_workshop.csv')

print(f"shape dos dados: {df.shape}")

df.head(10)

In [ ]:
#leitura direto da pasta - usando VSCode
df = pd.read_csv('../Database/dataset_workshop.csv', sep=';')

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
pd.set_option('display.max_rows', None)
df.dtypes

In [ ]:
# Criando a coluna de data
df['data'] = df['Date'].apply(lambda x: x.split(' ')[0])

In [ ]:
df['dur_aux'].fillna(0, inplace=True)

In [ ]:
# Modificando os dados
# Transformando a coluna DATA em fomrato datetime
# Retirando as vírgulas da coluna dur_aux e colocando pontos no lugar
# Transformando a coluna dur_aux em float

df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['dur_aux'] = df['dur_aux'].str.replace(',', '.')
df['dur_aux'] = df['dur_aux'].astype('float64')


In [ ]:
df['Satisfaction rating'].value_counts(dropna=False)

In [ ]:
# Retirando os -1 da coluna de Satisfação e colocando NaN no lugar (Not a Number)

df_sat = df.replace({'Satisfaction rating':-1}, np.nan)
df_sat['Satisfaction rating'].value_counts(dropna=False)

In [ ]:
# Sumarizando as colunas por data, usando diferentes agregadores

df_plot = df_sat[['data', 'Satisfaction rating', 'Call Id', 'Speed of Answer', 'dur_aux']].groupby('data').agg(
  {'Satisfaction rating':'mean',
  'Call Id':'count',
  'Speed of Answer':'sum',
  'dur_aux':'sum'}).reset_index()


In [ ]:
# Criando indicadores

df_plot['TM espera'] = df_plot['Speed of Answer'] / df_plot['Call Id']
df_plot['TM atendimento'] = df_plot['dur_aux'] / df_plot['Call Id']

In [ ]:
df_plot.head()

In [ ]:
df_plot.rename(columns={'Satisfaction rating':'satisfacao_cliente', 'Call Id':'total_ligacoes',
           'Speed of Answer':'tempo_para_atendimento', 'dur_aux':'duracao_atendimento'}, inplace=True)

In [ ]:
df_plot.head()

In [ ]:
fig = px.scatter(data_frame=df_plot, x='TM espera', y='satisfacao_cliente', trendline='ols', trendline_color_override='orange',
     title='Variação da Satisfação por Tempo de Espera', height=600)
fig.update_layout(xaxis_title='Tempo Médio de Espera', yaxis_title='Satisfação do Cliente')
fig.update_traces(marker={'size':8, 'line':{'width':1, 'color':'black'}}, line={'width':4})
fig.show()

In [ ]:
df_plot.head()

In [ ]:
px.scatter(data_frame=df_plot, x='total_ligacoes', y='satisfacao_cliente', width=600, height=500,
           trendline_color_override='orange', trendline='ols',
           title='Variação da Satisfação por Total de Ligações')

In [ ]:
px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500,
          title='Mapa de Correlação entre as Variáveis')

In [ ]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto=True,
          title='Mapa de Correlação entre as Variáveis')
fig.show()

In [ ]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto='.2f')
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
fig = px.imshow(df_plot.corr(), zmin=-1, zmax=1, width=600, height=500, text_auto='.2f', color_continuous_scale=['red', 'yellow', 'blue'])
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
px.line(data_frame=df_plot, x='data', y='satisfacao_cliente')

In [ ]:
px.line(data_frame=df_plot, x='data', y='satisfacao_cliente', hover_data='total_ligacoes')

In [ ]:
px.bar(data_frame=df_plot, x='data', y='total_ligacoes')

In [ ]:
px.bar(data_frame=df_plot, x='data', y='total_ligacoes', hover_data='satisfacao_cliente')

In [ ]:
from plotly.subplots import make_subplots

fig_line = px.line(data_frame=df_plot, x='data', y='satisfacao_cliente', hover_data='total_ligacoes')
fig_bar = px.bar(data_frame=df_plot, x='data', y='total_ligacoes', hover_data='satisfacao_cliente', color_discrete_sequence=['orange'])
fig = make_subplots(rows=2, cols=1, subplot_titles=('Satisfação', 'Total de Ligações'), shared_xaxes=True, vertical_spacing=0.1)

for x in fig_line['data']:
  fig.add_trace(x, row=1, col=1)
 
for figo in fig_bar['data']:
  fig.add_trace(figo, row=2, col=1)

fig.update_layout(height=600)
fig.show()